This is a cover letter builder using agentic ai


In [98]:
#imports

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel

In [99]:
load_dotenv(override=True)
openai = OpenAI()

In [100]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()
    

In [101]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [102]:
name = "Sviatoslav Rutkovskyi"


In [103]:
system_prompt = f"You are a proffesional cover letter writer, and your job is to write a cover letter for {name}, highlighting {name}'s skills, experience, and achievements. \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} in the letter as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, using the tone and style suitable for a cover letter.\
Do not make up any information, and only use the information provided.\
Respond with a cover letter and nothing else.\
Do not include the address or contact information. \
You will be given a job description, and you will need to tailor the cover letter to the job description."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"


In [104]:
# def requestLetter(message, history):
#     messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": message}]
#     response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
#     return response.choices[0].message.content

In [105]:



class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str
    

In [106]:
evaluator_system_prompt = f"You are an evaluator that decides whether a cover letter is acceptable. \
You are provided with {name}'s summary and LinkedIn profile, the job description, and the cover letter. \
Your task is to evaluate the cover letter, and reply with whether it is acceptable and your feedback. \
You need to ensure if the cover letter is professional, engaging, and tailored to the job description. \
You need to ensure if the cover letter was likely made by AI, and if it was made by AI, deny it, and provide feedback. \
You need to ensure if the cover letter includes the language present in the job description. \
You need to ensure that the cover letter has a strong and engaging opening paragraph. \
You need to ensure that the cover letter does not include any markdown and formatting besides the stardart 3 paragraph format. \
Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"

evaluator_system_prompt += f"With this context, please evaluate the cover letter, replying with whether the cover letter is acceptable and your feedback."

In [107]:
def evaluator_cover_letter(job_post, cover_letter):
    user_prompt = f"Here's the job posting presented by the user: \n\n{job_post}\n\n"
    user_prompt += f"Here's the cover letter generated by the agent: \n\n{cover_letter}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [108]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [109]:
def evaluate(job_post, cover_letter) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_cover_letter(job_post, cover_letter)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [110]:
def rerun(cover_letter, message, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous cover letter rejected\nYou just tried to create a cover letter, but the quality control rejected your cover letter\n"
    updated_system_prompt += f"## Your attempted cover letter:\n{cover_letter}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [111]:
import requests
from bs4 import BeautifulSoup

# ============================================================================
# METHOD 1: Simple HTTP Requests + BeautifulSoup (Most Common)
# ============================================================================

def scrape_webpage_simple(url):
    """
    Basic web scraping using requests and BeautifulSoup.
    Best for: Static HTML content, simple websites
    """
    try:
        # Set headers to mimic a real browser
        headers = {
            'User-Agent': 'Mozilla/5.0'
        }
        
        # Make the request
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Parse the HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        
        # Get all text content
        return soup.get_text()
        
        
    except requests.RequestException as e:
        return 'error'
    except Exception as e:
        return 'error'


In [112]:
def requestLetter(job_posting, history):
    page = scrape_webpage_simple(job_posting)
    print(page)
    if page == 'error':
        print("Failed to scrape job posting")
    else:
        job_posting = page

    messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": job_posting}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    cover_letter =response.choices[0].message.content

    evaluation = evaluate(cover_letter, job_posting)
    eval_counter = 0
    if eval_counter > 5:
        print("Failed evaluation - returning reply")
        return "Unable to generate cover letter" +"\n" + evaluation.feedback
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        eval_counter += 1
        print("Failed evaluation - retrying")
        # print(evaluation.feedback)
        cover_letter = rerun(cover_letter, job_posting,  evaluation.feedback)       
    return cover_letter

In [113]:
gr.ChatInterface(requestLetter, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


Java Engineer | Horizontal Talent Home/Search jobs/Java Engineer | Seattle, WAJava EngineerApply nowBack to job searchSeattle, WA|Onsite|Contract|Job ID 70656|Posted Jun 13, 2025JOB DESCRIPTIONJoin our team as a Software Development Engineer and play a pivotal role in enhancing our high-throughput communications platform. Your work will directly impact millions of users by delivering personalized messages at scale.
Responsibilities

Lead the migration of critical services from Java 8 to Java 21.
Design and develop scalable solutions for both front-end and back-end systems supporting mobile, web, and email communications.
Collaborate cross-functionally with product managers, scientists, and engineers to deliver high-quality solutions.
Help scale our platform to meet growing business demands.

Skills

Proficiency in Java and experience with Native AWS services.
Solid understanding of Unix/Linux environments.
Strong organizational skills and attention to detail.
Superior verbal and writte